# Forking out command line MPI tasks
Imagine I have a workflow that requires me to run an MPI executable (e.g., `mpiexec -n 4 <my_executable>`) and process the result. We can do that via the use of the `mpi_wrap()` function from the library, such that Dask manages the MPI runtime task that launches the job.
`mpi_wrap()` returns a dictionary:
```python
{
    "cmd": cmd_launched, 
    "out": stdout_output, 
    "err": stderr_output
}
```
This (currently) requires a little boilerplate code to work as one might expect (see the example below).

## Interacting with the task
If you need to grab information from the executed task, you can either do something to parse this dictionary, or interact with the executed task via the file system (i.e., read a result file).

In [ ]:
import os
from jobqueue_features.clusters import CustomSLURMCluster
from jobqueue_features.decorators import on_cluster, mpi_task
from jobqueue_features.mpi_wrapper import mpi_wrap
from jobqueue_features.functions import set_default_cluster

In [ ]:
set_default_cluster(CustomSLURMCluster)

This time when we declare our cluster we need to add a few additional keyword arguments:
* `mpi_mode` to let it know the cluster will use MPI
* `fork_mpi`, to indicate that we will be forking MPI processes
* `nodes` to indicate how much resources we want use (there are many other options here)

In [ ]:
custom_cluster = CustomSLURMCluster(
    name="mpiCluster",
    mpi_mode=True,
    fork_mpi=True,
    nodes=2,
)

In [ ]:
custom_cluster

We need boilerplate code because our decorators insert some kwargs (that define things like the number of processes) that are required for `mpi_wrap` to execute the task

In [ ]:
@on_cluster(cluster=custom_cluster)
@mpi_task(cluster=custom_cluster)
def mpi_wrap_task(**kwargs):
    return mpi_wrap(**kwargs)

And now we can use our newly created function `mpi_wrap_task()` in the same way we would use `mpi_wrap()`

In [ ]:
def forked_mpi():
    script_path = os.path.join("resources", "helloworld.py")
    t = mpi_wrap_task(
        executable="python", 
        exec_args=script_path
    )
    print("Ran\n\t", t.result()["cmd"])
    result = t.result()["out"]
    # Need to decode the output string so it is easily printed
    return result.decode('UTF-8')

In [ ]:
result = forked_mpi()

In [ ]:
print(result)